## Simple data analysis

Start by importing the required libraries

* `seaborn` = Python data visualization
* `matplotlib` = creating static, animated and interactive visualizations
* `os` = various operating system functionality
* `sys` = provides access to variables used by the interpreter


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys

We continue loading some utilities, feel free to browse the code located under `src/utils.py`

In [ ]:
sys.path.append(os.path.dirname(os.getcwd()))
from utils import (
    run_query,
    generate_sql_query,
    load_constants,
    explain_chart,
    generate_potential_questions
)

We call `load_constants()` function define the constants to be utilised

In [ ]:
constants = load_constants()

GOOGLE_CLOUD_PROJECT = constants["GCP"]["GOOGLE_CLOUD_PROJECT"]
GOOGLE_CLOUD_LOCATION = constants["GCP"]["GOOGLE_CLOUD_LOCATION"]
GOOGLE_CLOUD_GCS_BUCKET = constants["GCP"]["GOOGLE_CLOUD_GCS_BUCKET"]
GOOGLE_GEMINI_MODEL_15 = constants["VERTEX"]["GOOGLE_GEMINI_MODEL_15"]

GOOGLE_CLOUD_BIGQUERY_PROJECT = constants["BIGQUERY"]["GOOGLE_CLOUD_BIGQUERY_PROJECT"]
GOOGLE_CLOUD_BIGQUERY_DATASET = constants["BIGQUERY"]["GOOGLE_CLOUD_BIGQUERY_DATASET"]


BASE_TABLE_NAME_EVENTS = constants["BIGQUERY"]["BASE_TABLE_NAME_EVENTS"]
BASE_TABLE_NAME_INCIDENTS = constants["BIGQUERY"]["BASE_TABLE_NAME_INCIDENTS"]

### Inspect the base tables

Navigate to BigQuery, and inspect the tables `telco_rca_events` and `telco_rca_incidents` on the `rca_dataset`.

![data_analysis_03](../../assets/data_analysis_03.png)

### Query 1: Distance from average

Lets calculate the event count distance to the overall event average per each event type

In [ ]:
query = f"""WITH EventCounts AS (
    SELECT
        event,
        COUNT(*) AS event_count
    FROM
        `{GOOGLE_CLOUD_BIGQUERY_PROJECT}.{GOOGLE_CLOUD_BIGQUERY_DATASET}.{BASE_TABLE_NAME_EVENTS}`
    WHERE
        event IS NOT NULL
    GROUP BY
        event
),
AverageCount AS (
    SELECT
        AVG(event_count) AS average_count
    FROM
        EventCounts
)
SELECT
    ec.event,
    ec.event_count - ac.average_count AS difference_from_average
FROM
    EventCounts ec
CROSS JOIN
    AverageCount ac
ORDER BY
    difference_from_average;
    """

In [ ]:
df = run_query(query)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x="difference_from_average", y="event", data=df)
plt.title("Difference from Average by Event")
plt.xlabel("Difference from Average")
plt.ylabel("Event")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("diff_average.png")
plt.show()

Next, we will use Gemini API multimodal capablities to analyze the generated chart

In [ ]:
print(explain_chart(os.path.join(os.getcwd(),"diff_average.png"),
              GOOGLE_CLOUD_PROJECT,
              GOOGLE_CLOUD_LOCATION,
              GOOGLE_GEMINI_MODEL_15))

If you are more familiar with `pandas` instead of SQL, you can also use [BigQuery Dataframes](https://cloud.google.com/bigquery/docs/reference/bigquery-dataframes), an open source python package that "translates" from pandas dataframe syntax to BQ SQL

In [ ]:
import bigframes.pandas as bpd
bdf = bpd.read_gbq(f"{GOOGLE_CLOUD_BIGQUERY_PROJECT}.{GOOGLE_CLOUD_BIGQUERY_DATASET}.{BASE_TABLE_NAME_EVENTS}")

event_counts = bdf["event"].dropna().value_counts().reset_index()
event_counts.columns = ["event", "event_count"]
average_count = event_counts["event_count"].mean()

result = (
    event_counts.assign(
        difference_from_average=lambda x: x["event_count"] - average_count
    )
    .sort_values("difference_from_average", ascending=True)
    .reset_index(drop=True)
)

In [ ]:
result

### Query 2: Potential correlations

Lets find any potential correlations (Pearson) between the CPU Utilization and Memory Utilization on each type of network element to discard any potential side effects

In [ ]:
query = f"""SELECT 
  cpu_utilization.network_element_id,
  ROUND(CORR(cpu_utilization.value, memory_usage.value), 4) AS correlation_coefficient
      FROM    `{GOOGLE_CLOUD_BIGQUERY_PROJECT}.{GOOGLE_CLOUD_BIGQUERY_DATASET}.{BASE_TABLE_NAME_EVENTS}` cpu_utilization
      JOIN    `{GOOGLE_CLOUD_BIGQUERY_PROJECT}.{GOOGLE_CLOUD_BIGQUERY_DATASET}.{BASE_TABLE_NAME_EVENTS}` memory_usage 
  ON cpu_utilization.network_element_id = memory_usage.network_element_id 
  AND cpu_utilization.timestamp = memory_usage.timestamp
WHERE cpu_utilization.metric = 'CPU Utilization'  
  AND memory_usage.metric = 'Memory Utilization'
  AND cpu_utilization.value is not null 
  AND memory_usage.value is not null
GROUP BY network_element_id;
"""

In [ ]:
df = run_query(query)

In [ ]:
df

## Query 3: GenAI query

Next, we will use Gemini for a basic NL 2 SQL task

In [ ]:
query = generate_sql_query(
    "Calculate network elements with the most events",
    f"{GOOGLE_CLOUD_BIGQUERY_PROJECT}.{GOOGLE_CLOUD_BIGQUERY_DATASET}.{BASE_TABLE_NAME_EVENTS}",
    GOOGLE_CLOUD_PROJECT,
    GOOGLE_CLOUD_LOCATION,
    GOOGLE_GEMINI_MODEL_15
)

In [ ]:
print(query)

In [ ]:
run_query(query)

Try also with the direct integration right in the BigQuery Studio UI

![data_analysis_00](../../assets/data_analysis_00.png)

![data_analysis_01](../../assets/data_analysis_01.png)

![data_analysis_02](../../assets/data_analysis_02.png)

## Query 4: Try it yourself

Generate a query in both SQL and pandas , display a chart and explain it for the following business question:


In [ ]:
print(generate_potential_questions(f"{GOOGLE_CLOUD_BIGQUERY_PROJECT}.{GOOGLE_CLOUD_BIGQUERY_DATASET}.{BASE_TABLE_NAME_EVENTS}",
    GOOGLE_CLOUD_PROJECT,
    GOOGLE_CLOUD_LOCATION,
    GOOGLE_GEMINI_MODEL_15))